### New Datasets

- [ ] nRuaif/Long-instructions
- [ ] allenai/peer_read
- [ ] nampdn-ai/tiny-codes
- [ ] roneneldan/TinyStories
- [ ] sail/symbolic-instruction-tuning
- [ ] open-phi/textbooks

In [1]:
from datasets import concatenate_datasets, load_dataset, Dataset
from tqdm.auto import tqdm

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# source names
sources = [
    "nRuaif/Long-instructions",
    # "allenai/peer_read",
    "VatsaDev/TinyText",
    "nampdn-ai/tiny-codes",
    "roneneldan/TinyStories",
    "sail/symbolic-instruction-tuning",
    "open-phi/textbooks",
]

total_size = 250_000
size_per_dataset = total_size // len(sources)

datasets = {
    source: load_dataset(source).map(
        lambda x: dict(source=source)
    )
    for source in sources
}

datasets = {
    name: concatenate_datasets([
        Dataset.from_dict(
            dataset[split].shuffle(seed=42)[:size_per_dataset]
        )
        for split in list(dataset.keys())
    ])
    for name, dataset in datasets.items()
}


Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

In [4]:
dataset_list = list(datasets.values())

In [5]:
joined = concatenate_datasets(dataset_list).filter(lambda x: (
    any([v for k, v in x.items() if k != "source"])
))

Filter:   0%|          | 0/281935 [00:00<?, ? examples/s]

In [6]:
len(joined)

281931

In [7]:
joined.column_names

['text',
 'prompt',
 'completion',
 'source',
 'Text',
 'main_topic',
 'subtopic',
 'adjective',
 'action_verb',
 'scenario',
 'target_audience',
 'programming_language',
 'common_sense_topic',
 'idx',
 'response',
 'input',
 'output',
 'topic',
 'model',
 'concepts',
 'outline',
 'markdown',
 'field',
 'subfield',
 'rag']

In [21]:
next(a for a in tqdm(joined) if not (
    a["text"]
    or a["Text"]
    or (a["response"] and a["prompt"])
    or a["input"] # and a["output"]
    or (a["topic"] and a["markdown"])
))

  0%|          | 0/281931 [00:00<?, ?it/s]

StopIteration: 

In [9]:
d1, not_d1 = joined.filter(
    lambda x: x["text"]
).rename_column(
    "text", "target"
), joined.filter(
    lambda x: not x["text"]
)

d2, not_d2 = not_d1.filter(
    lambda x: x["Text"]
).rename_column(
    "Text", "target"
), not_d1.filter(
    lambda x: not x["Text"]
)

d3, not_d3 = not_d2.filter(
    lambda a: a["response"] and a["prompt"]
).map(
    lambda a: dict(target=a["prompt"] + " " + a["response"])
), not_d2.filter(
    lambda a: not (a["response"] and a["prompt"])
)

d4, not_d4 = not_d3.filter(
    lambda a: a["topic"] and a["markdown"]
).map(
    lambda a: dict(target=a["topic"] + "\n" + a["markdown"])
), not_d3.filter(
    lambda a: not (a["topic"] and a["markdown"])
)

d5 = not_d4.filter(
    lambda a: a["input"]
).map(
    lambda a: dict(target=a["input"] + (a["output"] or ""))
)

Filter:   0%|          | 0/281931 [00:00<?, ? examples/s]

Filter:   0%|          | 0/281931 [00:00<?, ? examples/s]

Filter:   0%|          | 0/176613 [00:00<?, ? examples/s]

Filter:   0%|          | 0/176613 [00:00<?, ? examples/s]

Filter:   0%|          | 0/134947 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/93281 [00:00<?, ? examples/s]

Map:   0%|          | 0/1795 [00:00<?, ? examples/s]

Filter:   0%|          | 0/93281 [00:00<?, ? examples/s]

Filter:   0%|          | 0/91486 [00:00<?, ? examples/s]

Map:   0%|          | 0/91486 [00:00<?, ? examples/s]

In [10]:
combined = concatenate_datasets([d1, d2, d3, d4, d5]).remove_columns(
    list(set(joined.column_names) - {"target", "source"})
).rename_column(
    "target", "response"
).map(lambda x: dict(prompt=""))

Map:   0%|          | 0/281931 [00:00<?, ? examples/s]

In [11]:
combined = combined.shuffle(seed=42).train_test_split(0.15)

In [12]:
combined.push_to_hub("diwank/samantha-pretraining-mix", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/7 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]